In [1]:
import pyodbc
import pandas as pd

# Define the function to fetch and preprocess data
def preprocessing_dataset():
    print("Connecting to SQL Server...")
    
    try:
        # Define connection string (replace with actual SQL Server details)
        conn = pyodbc.connect(
            "DRIVER={SQL Server};"
            "SERVER=10.12.30.240;"
            "DATABASE=GORPDWHBI;"  # Replace with your database name
            "UID=viewer;"
            "PWD=viewer1;"
        )
        print("Successfully connected to SQL Server!")

        # Query to fetch the required columns
        query = """
SELECT
b.TRANSACTIONID,
c.DeptName
FROM GORPDWH365.dbo.retailtransactionsalestrans a
LEFT JOIN GORPDWH365.dbo.RETAILTRANSACTIONTABLE b ON a.TRANSACTIONID = b.TRANSACTIONID
LEFT JOIN GORPDWHBI.dbo.DimProduct c on a.ITEMID = c.ItemID and a.DATAAREAID = c.DataAreaId
LEFT JOIN GORPDWHBI.dbo.DimCustomer d on a.CUSTACCOUNT = d.CustomerId and a.DATAAREAID = d.DataAreaId
LEFT JOIN GORPDWHBI.dbo.DimStore e on b.STORE = e.StoreId
--WAJIB
WHERE a.TRANSDATE BETWEEN '2024-01-01 00:00:00.000' AND '2024-12-31 00:00:00.000'
  and b.TYPE IN (2,19)
  and b.ENTRYSTATUS IN (0,2)
  and a.TRANSACTIONSTATUS IN (0,2)
  --WAJIB
  and c.DivName LIKE '%DIV BOOKS%'
  and c.DeptName IS NOT NULL
  and e.StoreName LIKE '%MARGONDA%'
        """  # Ganti 'your_table_name' dengan nama tabel yang benar
        print("Processing query...")

        # Fetch data into DataFrame
        market = pd.read_sql(query, conn)
        
        # Close connection
        conn.close()
        print("Successfully fetched data from SQL Server!")

        return market  # Make sure 'market' is always returned

    except Exception as e:
        print(f"Failed to connect or fetch data: {e}")
        return None  # Return None if there's an error

# Jalankan fungsi
market = preprocessing_dataset()

Connecting to SQL Server...
Successfully connected to SQL Server!
Processing query...


C:\Users\HP\AppData\Local\Temp\ipykernel_21148\4023732380.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  market = pd.read_sql(query, conn)


Successfully fetched data from SQL Server!


In [2]:
print(market) 

               TRANSACTIONID                     DeptName
0        10150-10150-12-5653                   DEP NOVELS
1        10150-10150-12-5653                   DEP NOVELS
2        10150-10150-12-5653  DEP RELIGION & SPIRITUALITY
3        10150-10150-12-5653                   DEP NOVELS
4        10150-10150-12-5653                   DEP COMICS
...                      ...                          ...
234387  10150-10150-06-14216                DEP CLEARANCE
234388  10150-10150-06-14114                DEP CLEARANCE
234389  10150-10150-06-17839                DEP CLEARANCE
234390  10150-10150-06-18253                DEP CLEARANCE
234391  10150-10150-06-18325                DEP CLEARANCE

[234392 rows x 2 columns]


In [4]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

# :pushpin: 1. Konversi data transaksi per TRANSACTIONID
transactions = market.groupby("TRANSACTIONID")["DeptName"].apply(list).tolist()

# :pushpin: 2. One-hot encoding
te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_array, columns=te.columns_).astype(bool)

# :pushpin: 3. Jalankan FP-Growth untuk menemukan pola item yang sering muncul bersama
frequent_itemsets = fpgrowth(df_encoded, min_support=0.00005, use_colnames=True, max_len=2)  # Atur min_support sesuai kebutuhan

# :pushpin: 4. Generate aturan asosiasi dengan batas minimum confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.00001)

# :pushpin: 5. Menggabungkan aturan yang saling berlawanan (A → B & B → A)
rekomendasi_rak = {}

if not rules.empty:
    for _, row in rules.iterrows():
        lhs = list(row['antecedents'])[0]  # Left Hand Side (LHS)
        rhs = list(row['consequents'])[0]  # Right Hand Side (RHS)
        support = row['support']
        confidence = row['confidence']
        lift = row['lift']

        # Buat pasangan unik (sorted agar A→B sama dengan B→A)
        pair_key = tuple(sorted([lhs, rhs]))

        # Simpan aturan dengan confidence tertinggi untuk setiap pasangan
        if pair_key not in rekomendasi_rak or confidence > rekomendasi_rak[pair_key][2]:
            rekomendasi_rak[pair_key] = (lhs, rhs, support, confidence, lift)

    # Urutkan berdasarkan Lift tertinggi
    sorted_rekomendasi = sorted(rekomendasi_rak.values(), key=lambda x: x[4], reverse=True)

    # Buat DataFrame hasil
    df_hasil = pd.DataFrame(sorted_rekomendasi, columns=["LHS", "RHS", "Support", "Confidence", "Lift"])

    # Simpan ke file Excel
    df_hasil.to_excel("Rekomendasi_Peletakan_Rak.xlsx", index=False)

    print(":white_check_mark: File 'Rekomendasi_Peletakan_Rak.xlsx' berhasil dibuat!")
else:
    print(":x: Tidak ada aturan yang ditemukan! Coba turunkan min_support atau min_threshold.")

:white_check_mark: File 'Rekomendasi_Peletakan_Rak.xlsx' berhasil dibuat!
